In [36]:
#ライブラリを自動リロード
%load_ext autoreload
%autoreload 2

import joblib
import pandas as pd


q_a_dict_path="develop/q_a_dict.bin"
text_dict=joblib.load(q_a_dict_path)
qa_df=pd.read_csv("develop/q_and_a_test.csv")


In [37]:
q_to_a_dict={}
for i in range(len(qa_df)):
    q=qa_df["Q"][i]
    a=qa_df["A"][i]
    q_to_a_dict[q]=a


In [62]:
import random

q_list=list(qa_df["Q"])
a_list=list(qa_df["A"])

q_a_dataset=[]
for q in q_list:
    q_a_dataset.append((q,q_to_a_dict[q],1))


for i in range(len(q_list)*1):
    q=random.choice(q_list)
    a=random.choice(a_list)
    if q_to_a_dict[q]!=a:
        q_a_dataset.append((q,a,-1))



In [63]:
#vectorize
data1=[]
data2=[]
label_list=[]
dataset_list=[]
for data in q_a_dataset:
    q=data[0]
    a=data[1]
    label=data[2]
    #v_q=text_dict[q].reshape(-1)
    #v_a=text_dict[a].reshape(-1)
    #data1.append(v_q)
    #data2.append(v_a)
    data1.append(q)
    data2.append(a)
    label_list.append(label)


In [64]:
from LLMSearch.Siamese.DictEmbedder import DictEmbedder
from LLMSearch.Embedding.GPTEmbedding import GPTEmbedding
from settings.key import GPT_API_KEY
gpt_embedder=GPTEmbedding(GPT_API_KEY)
embedder=DictEmbedder(q_a_dict_path,gpt_embedder)


In [94]:
from LLMSearch.Siamese.SiameseComparer import SiameseComparer
comparer=SiameseComparer(embedder)
comparer.prepare_model(data1,data2,label_list,hidden_dim=16)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                | Params
-----------------------------------------------------
0 | base_network | BaseNetwork         | 25.1 K
1 | loss         | CosineEmbeddingLoss | 0     
-----------------------------------------------------
25.1 K    Trainable params
0         Non-trainable params
25.1 K    Total params
0.101     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/kh/anaconda3/envs/lora/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/kh/anaconda3/envs/lora/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/kh/anaconda3/envs/lora/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (24) is smaller than the logging in

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [97]:
target_q="私は猫が好きです"
answers=["私は猫が好きです","猫は好きですか",a,"詩人の趣味は?","this is a pen","彼は怒っている"]

comparer.compare(
    [target_q]*len(answers),
    answers

                  )
#comparer.compare(q,a)

array([0.9999998 , 0.8280061 , 0.5429986 , 0.8624289 , 0.76487887,
       0.62774307], dtype=float32)

In [77]:
import matplotlib.pyplot as plt

0.008620689655172414